In [12]:
import numpy as np 
import pandas as pd
import pycountry

Se realizaran las normalizaciones de los nombres de paises generando nuevos csv para luego hacer los EDA

In [2]:
data_destino_2000 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_2000.csv')
data_origen_2000 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_2000.csv')

In [4]:
data_destino_2000.isna().sum()

Unnamed: 0                                    0
Region, development group, country or area    0
Location code                                 0
1990                                          0
1995                                          0
2000                                          0
2005                                          0
2010                                          0
2015                                          0
2020                                          0
Emigracion_Promedio                           0
Inmigracion_Promedio                          0
dtype: int64

In [5]:
data_destino_2000[data_destino_2000.duplicated()]

,Unnamed: 0,"Region, development group, country or area",Location code,1990,1995,2000,2005,2010,2015,2020,Emigracion_Promedio,Inmigracion_Promedio


In [6]:
data_origen_2000.isna().sum()

Unnamed: 0                                    0
Region, development group, country or area    0
Location code                                 0
1990                                          0
1995                                          0
2000                                          0
2005                                          0
2010                                          0
2015                                          0
2020                                          0
Emigracion_Promedio                           0
dtype: int64

In [7]:
data_origen_2000[data_origen_2000.duplicated()]

,Unnamed: 0,"Region, development group, country or area",Location code,1990,1995,2000,2005,2010,2015,2020,Emigracion_Promedio


In [8]:
data_origen_2000['Region, development group, country or area'] = data_origen_2000['Region, development group, country or area'].str.strip()
data_destino_2000['Region, development group, country or area'] = data_destino_2000['Region, development group, country or area'].str.strip()

In [9]:
data_origen_2000['Region, development group, country or area'] = data_origen_2000['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_2000['Region, development group, country or area'] = data_destino_2000['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})

In [10]:
data_destino_2000['Region, development group, country or area'].value_counts()

United States of America*     1
Russian Federation            1
Germany                       1
India                         1
France*                       1
Ukraine*                      1
Canada                        1
Saudi Arabia                  1
United Kingdom*               1
Australia*                    1
Pakistan                      1
Kazakhstan                    1
China, Hong Kong SAR*         1
Iran (Islamic Republic of)    1
United Arab Emirates          1
cote d'ivoire                 1
Name: Region, development group, country or area, dtype: int64

In [11]:
data_origen_2000['Region, development group, country or area'].value_counts()

United States of America*     1
Russian Federation            1
Germany                       1
India                         1
France*                       1
Ukraine*                      1
Canada                        1
Saudi Arabia                  1
United Kingdom*               1
Australia*                    1
Pakistan                      1
Kazakhstan                    1
China, Hong Kong SAR*         1
Iran (Islamic Republic of)    1
United Arab Emirates          1
cote d'ivoire                 1
Name: Region, development group, country or area, dtype: int64

observamos que hay caracteres especiales como ** procederemos a borrarlos 

In [13]:
def normalize_country(name):

    custom_mapping = {
        "Iran (Islamic Republic of)": "iran",
        "côte d'ivoire": "cote d'ivoire",
        "China, Hong Kong SAR*": "hong kong",
        "hong kong sar, china": "hong kong",
        "iran, islamic rep.": "iran"
    }
    
    if name in custom_mapping:
        return custom_mapping[name]
    try:
        country = pycountry.countries.lookup(name)
        return country.name
    except LookupError:
        if name == "United States of America*":
            return "United States"
        elif name == "Iran (Islamic Republic of)":
            return "iran"
        elif name == "Russian Federation":
            return "Russia"
        elif name == "France*":
            return "France"
        elif name == "Australia*":
            return "Australia"
        elif name == "United Kingdom*":
            return "United Kingdom"
        elif name == "China, Hong Kong SAR*":
            return "hong kong"                 
        elif name == "Ukraine*":
            return "Ukraine"
        # agrega más verificaciones aquí para otros casos especiales
        else:
          return name

In [15]:
#leemos la base de datos en este caso es WDI
data = pd.read_csv('../../datasets/processed/world_development_indicators/world_development_indicators.csv')

In [16]:
#creamos un codigo por pais
y = 0
data['cod'] = 0
l = []
for i in range(len(data)):
  if data['Country Code'].iloc[i] not in l:
    y += 1
    l.append(data['Country Code'].iloc[i])
    data['cod'].iloc[i] = y
  else:
    data['cod'].iloc[i] = y

C:\Users\USER\AppData\Local\Temp\ipykernel_15804\1947174363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cod'].iloc[i] = y
C:\Users\USER\AppData\Local\Temp\ipykernel_15804\1947174363.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cod'].iloc[i] = y


In [17]:
data_origen_2000['Region, development group, country or area'] = data_origen_2000['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_2000['Region, development group, country or area'] = data_destino_2000['Region, development group, country or area'].apply(normalize_country).str.lower()
data['Country Name'] = data['Country Name'].apply(normalize_country).str.lower()

In [18]:
filtro_or = data[(data['Country Name'].isin(data_origen_2000['Region, development group, country or area']))]
filtro_or.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-or-2000.csv', index = False)

filtro_de = data[(data['Country Name'].isin(data_destino_2000['Region, development group, country or area']))]
filtro_de.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-de-2000.csv', index = False)

In [19]:
data_origen_2005 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_2005.csv')
data_origen_2010 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_2010.csv')
data_origen_2015 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_2015.csv')
data_origen_2020 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_2020.csv')
data_destino_2005 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_2005.csv')
data_destino_2010 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_2010.csv')
data_destino_2015 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_2015.csv')
data_destino_2020 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_2020.csv')


In [20]:
data_origen_2005['Region, development group, country or area'] = data_origen_2005['Region, development group, country or area'].str.strip()
data_origen_2010['Region, development group, country or area'] = data_origen_2010['Region, development group, country or area'].str.strip()
data_origen_2015['Region, development group, country or area'] = data_origen_2015['Region, development group, country or area'].str.strip()
data_origen_2020['Region, development group, country or area'] = data_origen_2020['Region, development group, country or area'].str.strip()
data_destino_2005['Region, development group, country or area'] = data_destino_2005['Region, development group, country or area'].str.strip()
data_destino_2010['Region, development group, country or area'] = data_destino_2010['Region, development group, country or area'].str.strip()
data_destino_2015['Region, development group, country or area'] = data_destino_2015['Region, development group, country or area'].str.strip()
data_destino_2020['Region, development group, country or area'] = data_destino_2020['Region, development group, country or area'].str.strip()

In [21]:
data_origen_2005['Region, development group, country or area'] = data_origen_2005['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_origen_2010['Region, development group, country or area'] = data_origen_2010['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_origen_2015['Region, development group, country or area'] = data_origen_2015['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_origen_2020['Region, development group, country or area'] = data_origen_2020['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_2005['Region, development group, country or area'] = data_destino_2005['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_2010['Region, development group, country or area'] = data_destino_2010['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_2015['Region, development group, country or area'] = data_destino_2015['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_2020['Region, development group, country or area'] = data_destino_2020['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})

In [22]:
data_origen_2005['Region, development group, country or area'] = data_origen_2005['Region, development group, country or area'].apply(normalize_country).str.lower()
data_origen_2010['Region, development group, country or area'] = data_origen_2010['Region, development group, country or area'].apply(normalize_country).str.lower()
data_origen_2015['Region, development group, country or area'] = data_origen_2015['Region, development group, country or area'].apply(normalize_country).str.lower()
data_origen_2020['Region, development group, country or area'] = data_origen_2020['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_2005['Region, development group, country or area'] = data_destino_2005['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_2010['Region, development group, country or area'] = data_destino_2010['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_2015['Region, development group, country or area'] = data_destino_2015['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_2020['Region, development group, country or area'] = data_destino_2020['Region, development group, country or area'].apply(normalize_country).str.lower()

In [23]:
filtro_or1 = data[(data['Country Name'].isin(data_origen_2005['Region, development group, country or area']))]
filtro_or1.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-or-2005.csv', index = False)
filtro_or2 = data[(data['Country Name'].isin(data_origen_2010['Region, development group, country or area']))]
filtro_or2.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-or-2010.csv', index = False)
filtro_or3 = data[(data['Country Name'].isin(data_origen_2015['Region, development group, country or area']))]
filtro_or3.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2015-or.csv', index = False)
filtro_or4 = data[(data['Country Name'].isin(data_origen_2020['Region, development group, country or area']))]
filtro_or4.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2020-or.csv', index = False)

In [24]:
filtro_de1 = data[(data['Country Name'].isin(data_destino_2005['Region, development group, country or area']))]
filtro_de1.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2005-de.csv', index = False)
filtro_de2 = data[(data['Country Name'].isin(data_destino_2010['Region, development group, country or area']))]
filtro_de2.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2010-de.csv', index = False)
filtro_de3 = data[(data['Country Name'].isin(data_destino_2015['Region, development group, country or area']))]
filtro_de3.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2015-de.csv', index = False)
filtro_de4 = data[(data['Country Name'].isin(data_destino_2020['Region, development group, country or area']))]
filtro_de4.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-2020-de.csv', index = False)

In [26]:
data_origen_1990 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_1990.csv')
data_origen_1995 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_origen_1995.csv')
data_destino_1990 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_1990.csv')
data_destino_1995 = pd.read_csv('../../datasets/raw/proyecto_poblacion/top_16/top_16_destino_1995.csv')

In [27]:
data_origen_1990['Region, development group, country or area'] = data_origen_1990['Region, development group, country or area'].str.strip()
data_origen_1995['Region, development group, country or area'] = data_origen_1995['Region, development group, country or area'].str.strip()

In [28]:
data_destino_1990['Region, development group, country or area'] = data_destino_1990['Region, development group, country or area'].str.strip()
data_destino_1995['Region, development group, country or area'] = data_destino_1995['Region, development group, country or area'].str.strip()

In [29]:
data_origen_1990['Region, development group, country or area'] = data_origen_1990['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_origen_1995['Region, development group, country or area'] = data_origen_1995['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_1990['Region, development group, country or area'] = data_destino_1990['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
data_destino_1995['Region, development group, country or area'] = data_destino_1995['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})

In [30]:
data_origen_1990['Region, development group, country or area'] = data_origen_1990['Region, development group, country or area'].apply(normalize_country).str.lower()
data_origen_1995['Region, development group, country or area'] = data_origen_1995['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_1990['Region, development group, country or area'] = data_destino_1990['Region, development group, country or area'].apply(normalize_country).str.lower()
data_destino_1995['Region, development group, country or area'] = data_destino_1995['Region, development group, country or area'].apply(normalize_country).str.lower()

In [31]:
filtro_or5 = data[(data['Country Name'].isin(data_origen_1990['Region, development group, country or area']))]
filtro_or5.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-or-1990.csv', index = False)
filtro_or6 = data[(data['Country Name'].isin(data_origen_1995['Region, development group, country or area']))]
filtro_or6.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-or-1995.csv', index = False)

In [32]:
filtro_de5 = data[(data['Country Name'].isin(data_destino_1990['Region, development group, country or area']))]
filtro_de5.to_csv('../../datasets/processed/proyecto_poblacion/top_16/WDI-top-de-1990.csv', index = False)
filtro_de6 = data[(data['Country Name'].isin(data_destino_1995['Region, development group, country or area']))]
filtro_de6.to_csv('W../../datasets/processed/proyecto_poblacion/top_16/DI-top-de-1995.csv', index = False)